# NLP Sleep - Targeted Approach

## Methodology

### 1. Getting the NLP logic to work
The code didn't have all the right formats, but I was just trying to get the program to recognize the string patterns. It was only analyzing a string.

In [7]:
import re
import pandas as pd

# Sample function to extract AHI and ESS values based on observed patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Define regex patterns for Total AHI using various formats observed
    ahi_patterns = [
        r'AHI%:\s*([\d\.]+)',                    # Pattern: AHI%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)',       # Pattern: AHI: (total):: 11.1
        r'Overall AHI:\s*([\d\.]+)',             # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',    # Pattern: Total AHI: 11.1 per hour
        r'Total AHI:\s*([\d\.]+)',               # Pattern: Total AHI: 11.1
        r'AHI of\s*([\d\.]+)',                   # Pattern: AHI of 1.1
        r'pAHI%:\s*([\d\.]+)'                    # Pattern: pAHI%: 11.1
    ]

    # Define regex patterns for ESS using various formats observed
    ess_patterns = [
        r'ESS:\s*([\d]+)',                        # Pattern: ESS: 11
        r'EPWORTH:\s*([\d]+)',                    # Pattern: EPWORTH: 1
        r'ESS of\s*([\d]+)',                      # Pattern: ESS of 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)', # Pattern: Epworth Sleepiness Scale: 1
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)' # Pattern: EPWORTH SLEEPINESS SCALE: 1
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Example usage with sample narrative text
sample_text = "AHI of 12, and this is just a bunch of random text to make sure this works Epworth: 6"
total_ahi, ess = extract_ahi_ess(sample_text)
extract_ahi_ess
# Display the extracted values
total_ahi, ess

(12.0, 6)

### 2. Running the NLP Logic on a CSV, not a string
This was the first test running the program on a CSV. The program outputted every single column instead of just keeping columns with a value in AHI/ESS.

In [7]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Define regex patterns for Total AHI based on the given formats
    ahi_patterns = [
        r'AHI%:\s*([\d\.]+)',                    # Pattern: AHI%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)',       # Pattern: AHI: (total):: 11.1
        r'Overall AHI:\s*([\d\.]+)',             # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',    # Pattern: Total AHI: 11.1 per hour
        r'Total AHI:\s*([\d\.]+)',               # Pattern: Total AHI: 11.1
        r'AHI of\s*([\d\.]+)',                   # Pattern: AHI of 1.1
        r'pAHI%:\s*([\d\.]+)'                    # Pattern: pAHI%: 11.1
    ]

    # Define regex patterns for ESS based on the given formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                        # Pattern: ESS: 11
        r'EPWORTH:\s*([\d]+)',                    # Pattern: EPWORTH: 1
        r'ESS of\s*([\d]+)',                      # Pattern: ESS of 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)', # Pattern: Epworth Sleepiness Scale: 1
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)' # Pattern: EPWORTH SLEEPINESS SCALE: 1
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI and ESS values
def process_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if 'Narrative' column exists in the file
    if 'NARRATIVE' not in data.columns:
        print("Error: The CSV file must contain a 'Narrative' column with text data.")
        return
    
    # Initialize lists for extracted data
    extracted_data = {
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Append extracted values
        extracted_data["NARRATIVE"].append(narrative_text)
        extracted_data["Total AHI"].append(ahi_value)
        extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)
    
    # Save the extracted results to a new CSV file
    output_path = "extracted_ahi_ess_values.csv"
    extracted_df.to_csv(output_path, index=False)
    print(f"Extracted data saved to {output_path}")

# Example usage:
# Assuming the uploaded file is named 'sleep_study_data.csv'
file_path = '/Users/dennishwang/Desktop/NLP_sleep/95806A_6k_rows.csv'
process_csv(file_path)

Extracted data saved to extracted_ahi_ess_values.csv


In [20]:
out.head()

,NARRATIVE,Total AHI,ESS
0,Kaiser Permanente Sleep Laboratory,NaN,NaN
1,"9985 N. Sierra Avenue, Bldg 7 - Fontana, CA 92335",NaN,NaN
2,SLEEP ANALYSIS REPORT,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN


### 3. Removing all rows without a value in AHI/ESS columns
There was a ton of useless information keeping columns with narrative values like "Kaiser Permanente Sleep Laboratory" so I removed them and only kept rows that had a value in the Total AHI or ESS column.

In [16]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Define regex patterns for Total AHI based on the given formats
    ahi_patterns = [
        r'AHI%:\s*([\d\.]+)',                    # Pattern: AHI%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)',       # Pattern: AHI: (total):: 11.1
        r'Overall AHI:\s*([\d\.]+)',             # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',    # Pattern: Total AHI: 11.1 per hour
        r'Total AHI:\s*([\d\.]+)',               # Pattern: Total AHI: 11.1
        r'AHI of\s*([\d\.]+)',                   # Pattern: AHI of 1.1
        r'pAHI%:\s*([\d\.]+)'                    # Pattern: pAHI%: 11.1
    ]

    # Define regex patterns for ESS based on the given formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                        # Pattern: ESS: 11
        r'EPWORTH:\s*([\d]+)',                    # Pattern: EPWORTH: 1
        r'ESS of\s*([\d]+)',                      # Pattern: ESS of 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)', # Pattern: Epworth Sleepiness Scale: 1
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)' # Pattern: EPWORTH SLEEPINESS SCALE: 1
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI and ESS values
def process_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if 'Narrative' column exists in the file
    if 'NARRATIVE' not in data.columns:
        print("Error: The CSV file must contain a 'NARRATIVE' column with text data.")
        return
    
    # Initialize lists for extracted data
    extracted_data = {
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Append extracted values
        extracted_data["NARRATIVE"].append(narrative_text)
        extracted_data["Total AHI"].append(ahi_value)
        extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)
    
    # Remove rows where both 'Total AHI' and 'ESS' are NaN
    extracted_df.dropna(subset=["Total AHI", "ESS"], how='all', inplace=True)
    
    # Save the filtered extracted results to a new CSV file
    output_path = "extracted_ahi_ess_values.csv"
    extracted_df.to_csv(output_path, index=False)
    print(f"Extracted data saved to {output_path}")

# Example usage:
# Assuming the uploaded file is named 'sleep_study_data.csv'
file_path = '/Users/dennishwang/Desktop/NLP_sleep/95806A_6k_rows.csv'
process_csv(file_path)

Extracted data saved to extracted_ahi_ess_values.csv


In [19]:
out_df = pd.read_csv('extracted_ahi_ess_values.csv')
out_df.head()

,NARRATIVE,Total AHI,ESS
0,Epworth: 12/24,NaN,12.0
1,Epworth: 11/24,NaN,11.0
2,Epworth: 15/24,NaN,15.0
3,Epworth: 5/24,NaN,5.0
4,Total AHI: 38.3 per hr (Obstructive Index34 ...,38.3,NaN


### 4. Adding missing formats and outputting MRN and PROC_DATE
I outputted the MRN and PROC_DATE of each row because it makes it a lot easier to check the accuracy of the program. I also realized that many values were missing because ChatGPT didn't generate all of the formats I provided.

In [25]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Define AHI patterns based on the given formats
    ahi_patterns = [
        r'AHI4%:\s*([\d\.]+)',                            # Pattern: AHI4%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)\s*events/hour',  # Pattern: AHI: (total):: 11.1 events/hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',             # Pattern: Total AHI: 11.1 per hour
        r'pAHI:\s*([\d\.]+)',                             # Pattern: pAHI: 1.1
        r'AHI of\s*([\d\.]+)',                            # Pattern: AHI of 1.1
        r'pAHI4%:\s*([\d\.]+)',                           # Pattern: pAHI4%:1.11
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Overall AHI:\s*([\d\.]+)\s*per hour',           # Pattern: Overall AHI: 11.1 per hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)',                        # Pattern: Total AHI: 11.1
        r'Total AHI4%:\s*([\d\.]+)'                       # Pattern: Total AHI4%: 11.1
    ]

    # Define ESS patterns based on the new formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                               # Pattern: ESS: 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)(?:\s*out of\s*\d+)?',  # Pattern: Epworth Sleepiness Scale: 11 out of 24
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)',       # Pattern: EPWORTH SLEEPINESS SCALE: 11
        r'Patient reports ESS of\s*([\d]+)',             # Pattern: Patient reports ESS of 11
        r'EPWORTH:\s*([\d]+)',                           # Pattern: EPWORTH: 11
        r'Epworth:\s*([\d]+)(?:/\d+)?',                  # Pattern: Epworth: 11 or Epworth: 11/24
        r'Epworth Score:\s*([\d]+)'                      # Pattern: Epworth Score: 11
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI, ESS, MRN, and PROC_DATE values
def process_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if necessary columns exist in the file
    required_columns = ['NARRATIVE', 'MRN', 'PROC_DATE']
    for col in required_columns:
        if col not in data.columns:
            print(f"Error: The CSV file must contain a '{col}' column.")
            return
    
    # Initialize lists for extracted data
    extracted_data = {
        "MRN": [],
        "PROC_DATE": [],
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        mrn = row['MRN']
        proc_date = row['PROC_DATE']
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Append extracted values
        extracted_data["MRN"].append(mrn)
        extracted_data["PROC_DATE"].append(proc_date)
        extracted_data["NARRATIVE"].append(narrative_text)
        extracted_data["Total AHI"].append(ahi_value)
        extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)
    
    # Remove rows where both 'Total AHI' and 'ESS' are NaN
    extracted_df.dropna(subset=["Total AHI", "ESS"], how='all', inplace=True)
    
    # Save the filtered extracted results to a new CSV file
    output_path = "extract_1.csv"
    extracted_df.to_csv(output_path, index=False)
    print(f"Extracted data saved to {output_path}")

# Example usage:
# Assuming the uploaded file is named 'sleep_study_data.csv'
file_path = '/Users/dennishwang/Desktop/NLP_sleep/95806A_6k_rows.csv'
process_csv(file_path)


Extracted data saved to extract_1.csv


In [24]:
extract_1 = pd.read_csv('extract_1.csv')
extract_1.head()

,MRN,PROC_DATE,NARRATIVE,Total AHI,ESS
0,8410,31MAR2010:00:00:00,Total AHI4%: 21.1 (131 minutes),21.1,NaN
1,8410,31MAR2010:00:00:00,Supine AHI4%: 32.6 (97 minutes),32.6,NaN
2,8410,31MAR2010:00:00:00,Non-Supine AHI4%: 10.5 (34 minutes),10.5,NaN
3,8410,31MAR2010:00:00:00,Epworth: 12/24,NaN,12.0
4,32809,26JUL2010:00:00:00,Total AHI4%: 2.0 (10 minutes),2.0,NaN


### 5. Removing supine/non-supine values
Supine and non-supine values were confusing the program and making it output inaccurate AHI values. To solve this, if the program reads Supine/Non-Supine, it skips the row.

In [30]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Check for "Supine" or "Non-Supine" preceding patterns to skip extraction
    if re.search(r'\b(Supine|Non-Supine)\b', text, re.IGNORECASE):
        return ahi_value, ess_value  # Return None for both, skipping the row

    # Define AHI patterns based on the given formats
    ahi_patterns = [
        r'AHI4%:\s*([\d\.]+)',                            # Pattern: AHI4%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)\s*events/hour',  # Pattern: AHI: (total):: 11.1 events/hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',             # Pattern: Total AHI: 11.1 per hour
        r'pAHI:\s*([\d\.]+)',                             # Pattern: pAHI: 1.1
        r'AHI of\s*([\d\.]+)',                            # Pattern: AHI of 1.1
        r'pAHI4%:\s*([\d\.]+)',                           # Pattern: pAHI4%:1.11
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Overall AHI:\s*([\d\.]+)\s*per hour',           # Pattern: Overall AHI: 11.1 per hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)',                        # Pattern: Total AHI: 11.1
        r'Total AHI4%:\s*([\d\.]+)'                       # Pattern: Total AHI4%: 11.1
    ]

    # Define ESS patterns based on the new formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                               # Pattern: ESS: 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)(?:\s*out of\s*\d+)?',  # Pattern: Epworth Sleepiness Scale: 11 out of 24
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)',       # Pattern: EPWORTH SLEEPINESS SCALE: 11
        r'Patient reports ESS of\s*([\d]+)',             # Pattern: Patient reports ESS of 11
        r'EPWORTH:\s*([\d]+)',                           # Pattern: EPWORTH: 11
        r'Epworth:\s*([\d]+)(?:/\d+)?',                  # Pattern: Epworth: 11 or Epworth: 11/24
        r'Epworth Score:\s*([\d]+)'                      # Pattern: Epworth Score: 11
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI, ESS, MRN, and PROC_DATE values
def process_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if necessary columns exist in the file
    required_columns = ['NARRATIVE', 'MRN', 'PROC_DATE']
    for col in required_columns:
        if col not in data.columns:
            print(f"Error: The CSV file must contain a '{col}' column.")
            return
    
    # Initialize lists for extracted data
    extracted_data = {
        "MRN": [],
        "PROC_DATE": [],
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        mrn = row['MRN']
        proc_date = row['PROC_DATE']
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Only append rows where at least one of AHI or ESS values is extracted
        if ahi_value is not None or ess_value is not None:
            extracted_data["MRN"].append(mrn)
            extracted_data["PROC_DATE"].append(proc_date)
            extracted_data["NARRATIVE"].append(narrative_text)
            extracted_data["Total AHI"].append(ahi_value)
            extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)
    
    # Remove rows where both 'Total AHI' and 'ESS' are NaN
    extracted_df.dropna(subset=["Total AHI", "ESS"], how='all', inplace=True)
    
    # Save the filtered extracted results to a new CSV file
    output_path = "extract_3.csv"
    extracted_df.to_csv(output_path, index=False)
    print(f"Extracted data saved to {output_path}")

# Example usage:
# Assuming the uploaded file is named 'sleep_study_data.csv'
file_path = '/Users/dennishwang/Desktop/NLP_sleep/95806A_6k_rows.csv'
process_csv(file_path)


Extracted data saved to extract_3.csv


In [31]:
extract_3 = pd.read_csv('extract_3.csv')
extract_3.head()

,MRN,PROC_DATE,NARRATIVE,Total AHI,ESS
0,8410,31MAR2010:00:00:00,Total AHI4%: 21.1 (131 minutes),21.1,NaN
1,8410,31MAR2010:00:00:00,Epworth: 12/24,NaN,12.0
2,32809,26JUL2010:00:00:00,Total AHI4%: 2.0 (10 minutes),2.0,NaN
3,32809,26JUL2010:00:00:00,Epworth: 11/24,NaN,11.0
4,101357,09AUG2010:00:00:00,Total AHI4%: 64.3 (465 minutes),64.3,NaN


### 6. Keeping rows with NaN values, combining MRN, and creating "conversion_error"
I combined the same MRN so we can see an MRN and its respective Total AHI/ESS. I also chose to leave rows with NaN values to see whether the program was messing up or if there actually was no value. When running the program on the entire dataset, there were certain strings the program had issues translating into floats. I replaced those strings initially with a NaN value, but switched it to "conversion_error" to be more descriptive and make sure it isn't confused with a missing AHI/ESS value.

In [5]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Check for "Supine" or "Non-Supine" preceding patterns to skip extraction
    if re.search(r'\b(Supine|Non-Supine)\b', text, re.IGNORECASE):
        return ahi_value, ess_value  # Return None for both, skipping the row

    # Define AHI patterns based on the given formats
    ahi_patterns = [
        r'AHI4%:\s*([\d\.]+)',                            # Pattern: AHI4%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)\s*events/hour',  # Pattern: AHI: (total):: 11.1 events/hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',             # Pattern: Total AHI: 11.1 per hour
        r'pAHI:\s*([\d\.]+)',                             # Pattern: pAHI: 1.1
        r'AHI of\s*([\d\.]+)',                            # Pattern: AHI of 1.1
        r'pAHI4%:\s*([\d\.]+)',                           # Pattern: pAHI4%:1.11
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Overall AHI:\s*([\d\.]+)\s*per hour',           # Pattern: Overall AHI: 11.1 per hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)',                        # Pattern: Total AHI: 11.1
        r'Total AHI4%:\s*([\d\.]+)'                       # Pattern: Total AHI4%: 11.1
    ]

    # Define ESS patterns based on the new formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                               # Pattern: ESS: 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)(?:\s*out of\s*\d+)?',  # Pattern: Epworth Sleepiness Scale: 11 out of 24
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)',       # Pattern: EPWORTH SLEEPINESS SCALE: 11
        r'Patient reports ESS of\s*([\d]+)',             # Pattern: Patient reports ESS of 11
        r'EPWORTH:\s*([\d]+)',                           # Pattern: EPWORTH: 11
        r'Epworth:\s*([\d]+)(?:/\d+)?',                  # Pattern: Epworth: 11 or Epworth: 11/24
        r'Epworth Score:\s*([\d]+)'                      # Pattern: Epworth Score: 11
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_value = float(match.group(1))
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_value = int(match.group(1))
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI, ESS, MRN, and PROC_DATE values, with MRN consolidation
def process_csv(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if necessary columns exist in the file
    required_columns = ['NARRATIVE', 'MRN', 'PROC_DATE']
    for col in required_columns:
        if col not in data.columns:
            print(f"Error: The CSV file must contain a '{col}' column.")
            return
    
    # Initialize lists for extracted data
    extracted_data = {
        "MRN": [],
        "PROC_DATE": [],
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        mrn = row['MRN']
        proc_date = row['PROC_DATE']
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Append all rows with or without extracted values
        extracted_data["MRN"].append(mrn)
        extracted_data["PROC_DATE"].append(proc_date)
        extracted_data["NARRATIVE"].append(narrative_text if ahi_value or ess_value else None)
        extracted_data["Total AHI"].append(ahi_value)
        extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)

    # Consolidate rows with the same MRN
    consolidated_df = extracted_df.groupby("MRN").agg({
        "PROC_DATE": lambda x: ', '.join(map(str, x.unique())),               # Combine unique dates
        "NARRATIVE": lambda x: '; '.join(filter(None, x)),                    # Combine narratives
        "Total AHI": "first",                                                 # Use first non-NaN AHI
        "ESS": "first"                                                        # Use first non-NaN ESS
    }).reset_index()

    # Ensure that even MRNs without AHI or ESS are kept, with NaNs if necessary
    consolidated_df["Total AHI"] = consolidated_df["Total AHI"].fillna("NaN")
    consolidated_df["ESS"] = consolidated_df["ESS"].fillna("NaN")
    consolidated_df["NARRATIVE"] = consolidated_df["NARRATIVE"].apply(lambda x: x if x else "NaN")

    # Save the consolidated extracted results to a new CSV file
    output_path = "extract_5.csv"
    consolidated_df.to_csv(output_path, index=False)
    print(f"Consolidated data saved to {output_path}")

# Example usage:
# Assuming the uploaded file is named 'sleep_study_data.csv'
file_path = '/Users/dennishwang/Desktop/NLP_sleep/95806A_6k_rows.csv'
process_csv(file_path)


Consolidated data saved to extract_5.csv


In [34]:
extract_5 = pd.read_csv('extract_5.csv')
extract_5.head()

,MRN,PROC_DATE,NARRATIVE,Total AHI,ESS
0,3646,16NOV2015:00:00:00,NaN,NaN,NaN
1,8410,31MAR2010:00:00:00,Total AHI4%: 21.1 (131 minutes);...,21.1,12.0
2,17844,04APR2016:00:00:00,Total AHI4%: 11.1; AHI4%: 11.1; ODI4%: 16.1; T...,11.1,11.0
3,17895,13JAN2011:00:00:00,Neck Circumference: 19.5 inch. Epworth Score...,44.8,3.0
4,18652,23JUL2015:00:00:00,Total AHI4%: 72.6; AHI4%: 72.6 ODI4%: 58.5 T90...,72.6,10.0


## Final Version (Keep all MRNs)

In [16]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns with validation
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Check for "Supine" or "Non-Supine" preceding patterns to skip extraction
    if re.search(r'\b(Supine|Non-Supine)\b', text, re.IGNORECASE):
        return ahi_value, ess_value  # Return None for both, skipping the row

    # Define AHI patterns based on the given formats
    ahi_patterns = [
        r'AHI4%:\s*([\d\.]+)',                            # Pattern: AHI4%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)\s*events/hour',  # Pattern: AHI: (total):: 11.1 events/hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',             # Pattern: Total AHI: 11.1 per hour
        r'pAHI:\s*([\d\.]+)',                             # Pattern: pAHI: 1.1
        r'AHI of\s*([\d\.]+)',                            # Pattern: AHI of 1.1
        r'pAHI4%:\s*([\d\.]+)',                           # Pattern: pAHI4%:1.11
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Overall AHI:\s*([\d\.]+)\s*per hour',           # Pattern: Overall AHI: 11.1 per hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)',                        # Pattern: Total AHI: 11.1
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Apnea / Hypopnea Index\s*([\d\.]+)',            # New: Apnea / Hypopnea Index 16.6
        r'AHI\s*([\d\.]+)\s*(?:REM AHI [\d\.]+)?\s*(?:RDI [\d\.]+)?', # New: AHI 43.9 (ignore REM AHI and RDI)
        r'Respirations:\s*AHI4%\s*([\d\.]+)\s*\(supine [\d\.]+;\s*non-supine [\d\.]+\)', # New: Respirations AHI4%
    ]

    # Define ESS patterns with the new format
    ess_patterns = [
        r'ESS:\s*([\d]+)',                               # Pattern: ESS: 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)(?:\s*out of\s*\d+)?',  # Pattern: Epworth Sleepiness Scale: 11 out of 24
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)',       # Pattern: EPWORTH SLEEPINESS SCALE: 11
        r'Patient reports ESS of\s*([\d]+)',             # Pattern: Patient reports ESS of 11
        r'EPWORTH:\s*([\d]+)',                           # Pattern: EPWORTH: 11
        r'Epworth:\s*([\d]+)(?:/\d+)?',                  # Pattern: Epworth: 11 or Epworth: 11/24
        r'Epworth Score:\s*([\d]+)',                     # Pattern: Epworth Score: 11
        r'Epworth\s*([\d]+)(?:/\d+)?'                    # New: Epworth 10/24
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_candidate = match.group(1)
            # Validate that ahi_candidate is a valid number (no multiple decimal points)
            if re.match(r'^\d+(\.\d+)?$', ahi_candidate):  # Matches integers or floats with only one decimal
                ahi_value = float(ahi_candidate)
            else:
                ahi_value = "conversion_error"
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_candidate = match.group(1)
            if re.match(r'^\d+$', ess_candidate):  # Matches integers only
                ess_value = int(ess_candidate)
            else:
                ess_value = "conversion_error"
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI, ESS, MRN, and PROC_DATE values, keeping all rows
def process_csv_keep_all(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if necessary columns exist in the file
    required_columns = ['NARRATIVE', 'MRN', 'PROC_DATE']
    for col in required_columns:
        if col not in data.columns:
            print(f"Error: The CSV file must contain a '{col}' column.")
            return
    
    # Initialize lists for extracted data
    extracted_data = {
        "MRN": [],
        "PROC_DATE": [],
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        mrn = row['MRN']
        proc_date = row['PROC_DATE']
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Append all rows with or without extracted values
        extracted_data["MRN"].append(mrn)
        extracted_data["PROC_DATE"].append(proc_date)
        extracted_data["NARRATIVE"].append(narrative_text)
        extracted_data["Total AHI"].append(ahi_value)
        extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)

    # Consolidate rows with the same MRN
    consolidated_df = extracted_df.groupby("MRN").agg({
        "PROC_DATE": lambda x: ', '.join(map(str, x.unique())),               # Combine unique dates
        "NARRATIVE": lambda x: '; '.join(filter(None, x)),                    # Combine narratives
        "Total AHI": "first",                                                 # Use first non-NaN AHI or conversion_error
        "ESS": "first"                                                        # Use first non-NaN ESS or conversion_error
    }).reset_index()

    # Save the consolidated extracted results to a new CSV file
    output_path = "final_all_mrn.csv"
    consolidated_df.to_csv(output_path, index=False)
    print(f"Consolidated data with all rows saved to {output_path}")

# Example usage for all rows:
file_path = '/Users/dennishwang/Desktop/NLP_sleep/sleepstudies.csv'
process_csv_keep_all(file_path)


Consolidated data with all rows saved to final_all_mrn.csv


In [36]:
final_all_mrn = pd.read_csv('final_all_mrn.csv')
final_all_mrn.head()

,MRN,PROC_DATE,NARRATIVE,Total AHI,ESS
0,3646,16NOV2015:00:00:00,NaN,NaN,NaN
1,8410,31MAR2010:00:00:00,Name: Frank Horzen; Date of Birth: 30.12.1942;...,21.1,12.0
2,10011,12FEB2008:00:00:00,Kaiser Permanente Sleep Laboratory; 9985 N. Si...,8.9,NaN
3,15778,15DEC2014:00:00:00,Ambulatory Sleep Study; Study conducted as an ...,18.5,NaN
4,17844,04APR2016:00:00:00,"PATIENT INFORMATION; nan; Name: Medina, Julian...",11.1,11.0


## Remove MRNs with all NaNs
Removed every row where Total_AHI = NaN and ESS = NaN

In [33]:
import re
import pandas as pd

# Function to extract AHI and ESS values based on patterns with validation
def extract_ahi_ess(text):
    ahi_value, ess_value = None, None

    # Check for "Supine" or "Non-Supine" preceding patterns to skip extraction
    if re.search(r'\b(Supine|Non-Supine)\b', text, re.IGNORECASE):
        return ahi_value, ess_value  # Return None for both, skipping the row

    # Define AHI patterns based on the given formats
    ahi_patterns = [
        r'AHI4%:\s*([\d\.]+)',                            # Pattern: AHI4%: 11.1
        r'AHI:\s*\(total\)::\s*([\d\.]+)\s*events/hour',  # Pattern: AHI: (total):: 11.1 events/hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)\s*per hour',             # Pattern: Total AHI: 11.1 per hour
        r'pAHI:\s*([\d\.]+)',                             # Pattern: pAHI: 1.1
        r'AHI of\s*([\d\.]+)',                            # Pattern: AHI of 1.1
        r'pAHI4%:\s*([\d\.]+)',                           # Pattern: pAHI4%:1.11
        r'Total AHI4%:\s*([\d\.]+)',                      # Pattern: Total AHI4%: 11.1
        r'Overall AHI:\s*([\d\.]+)\s*per hour',           # Pattern: Overall AHI: 11.1 per hour
        r'Overall AHI:\s*([\d\.]+)',                      # Pattern: Overall AHI: 11.1
        r'Total AHI:\s*([\d\.]+)',                        # Pattern: Total AHI: 11.1
        r'Total AHI4%:\s*([\d\.]+)'                       # Pattern: Total AHI4%: 11.1
    ]

    # Define ESS patterns based on the new formats
    ess_patterns = [
        r'ESS:\s*([\d]+)',                               # Pattern: ESS: 11
        r'Epworth Sleepiness Scale\s*:\s*([\d]+)(?:\s*out of\s*\d+)?',  # Pattern: Epworth Sleepiness Scale: 11 out of 24
        r'EPWORTH SLEEPINESS SCALE\s*:\s*([\d]+)',       # Pattern: EPWORTH SLEEPINESS SCALE: 11
        r'Patient reports ESS of\s*([\d]+)',             # Pattern: Patient reports ESS of 11
        r'EPWORTH:\s*([\d]+)',                           # Pattern: EPWORTH: 11
        r'Epworth:\s*([\d]+)(?:/\d+)?',                  # Pattern: Epworth: 11 or Epworth: 11/24
        r'Epworth Score:\s*([\d]+)'                      # Pattern: Epworth Score: 11
    ]

    # Search through AHI patterns to find a match
    for pattern in ahi_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ahi_candidate = match.group(1)
            # Validate that ahi_candidate is a valid number (no multiple decimal points)
            if re.match(r'^\d+(\.\d+)?$', ahi_candidate):  # Matches integers or floats with only one decimal
                ahi_value = float(ahi_candidate)
            else:
                ahi_value = "conversion_error"
            break

    # Search through ESS patterns to find a match
    for pattern in ess_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            ess_candidate = match.group(1)
            if re.match(r'^\d+$', ess_candidate):  # Matches integers only
                ess_value = int(ess_candidate)
            else:
                ess_value = "conversion_error"
            break

    return ahi_value, ess_value

# Function to process a CSV file and extract AHI, ESS, MRN, and PROC_DATE values, excluding rows without AHI or ESS
def process_csv_exclude_empty(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path)
    
    # Check if necessary columns exist in the file
    required_columns = ['NARRATIVE', 'MRN', 'PROC_DATE']
    for col in required_columns:
        if col not in data.columns:
            print(f"Error: The CSV file must contain a '{col}' column.")
            return
    
    # Initialize lists for extracted data
    extracted_data = {
        "MRN": [],
        "PROC_DATE": [],
        "NARRATIVE": [],
        "Total AHI": [],
        "ESS": []
    }
    
    # Process each row in the CSV file
    for index, row in data.iterrows():
        narrative_text = str(row['NARRATIVE'])
        mrn = row['MRN']
        proc_date = row['PROC_DATE']
        ahi_value, ess_value = extract_ahi_ess(narrative_text)
        
        # Only append rows with at least one valid AHI or ESS value
        if ahi_value is not None or ess_value is not None:
            extracted_data["MRN"].append(mrn)
            extracted_data["PROC_DATE"].append(proc_date)
            extracted_data["NARRATIVE"].append(narrative_text)
            extracted_data["Total AHI"].append(ahi_value)
            extracted_data["ESS"].append(ess_value)
    
    # Create a DataFrame with the extracted results
    extracted_df = pd.DataFrame(extracted_data)

    # Consolidate rows with the same MRN
    consolidated_df = extracted_df.groupby("MRN").agg({
        "PROC_DATE": lambda x: ', '.join(map(str, x.unique())),               # Combine unique dates
        "NARRATIVE": lambda x: '; '.join(filter(None, x)),                    # Combine narratives
        "Total AHI": "first",                                                 # Use first non-NaN AHI or conversion_error
        "ESS": "first"                                                        # Use first non-NaN ESS or conversion_error
    }).reset_index()

    # Save the consolidated extracted results to a new CSV file
    output_path = "final_no_nan_mrn.csv"
    consolidated_df.to_csv(output_path, index=False)
    print(f"Consolidated data (excluding rows without AHI or ESS) saved to {output_path}")

# Example usage for excluding rows without AHI or ESS:
file_path = '/Users/dennishwang/Desktop/NLP_sleep/sleepstudies.csv'
process_csv_exclude_empty(file_path)


Consolidated data (excluding rows without AHI or ESS) saved to final_no_nan_mrn.csv


In [35]:
final_no_nan_mrn = pd.read_csv('final_no_nan_mrn.csv')
final_no_nan_mrn.head()

,MRN,PROC_DATE,NARRATIVE,Total AHI,ESS
0,8410,31MAR2010:00:00:00,Total AHI4%: 21.1 (131 minutes);...,21.1,12.0
1,15778,15DEC2014:00:00:00,Overall AHI: 18.5,18.5,NaN
2,17844,04APR2016:00:00:00,Total AHI4%: 11.1; AHI4%: 11.1; ODI4%: 16.1; T...,11.1,11.0
3,17895,13JAN2011:00:00:00,Neck Circumference: 19.5 inch. Epworth Score...,44.8,3.0
4,18652,23JUL2015:00:00:00,Total AHI4%: 72.6; AHI4%: 72.6 ODI4%: 58.5 T90...,72.6,10.0
